# Transfer Learning for Plant Disease Classification using TensorFlow

## Overview
This case study demonstrates transfer learning techniques for plant disease classification using the PlantVillage dataset. Students will gain hands-on experience developing an image classification model using TensorFlow and transfer learning.

## Learning Objectives

*   Understand transfer learning and its benefits in image classification tasks.
*   Learn to load and preprocess image data using TensorFlow Datasets (TFDS).
*   Use a pre-trained model (MobileNetV2) as a base model, add custom layers, and fine-tune it for improved performance.
*   Evaluate the model's performance and interpret the results.

## Project Description

The PlantVillage dataset contains 38 classes of healthy and diseased plant leaves. You will develop a deep-learning model to classify images based on plant diseases. There are the steps:

1.  Load the dataset using TFDS and preprocess the images.
2. Set input image shape, number of classes, and batch size.
3. Load the pre-trained MobileNetV2 model and add custom layers.
4. Create the final model and freeze the base model layers.
5. Compile, train, and fine-tune the model.
6. Evaluate the model's performance on the test dataset.

You will experiment with different parameters and architectures to achieve optimal performance while better understanding transfer learning, TensorFlow, and image classification techniques.




## Setup

Set directory path

In [1]:
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# # change directory
# os.chdir('/content/drive/My Drive/EM0007/1. Supervised/2. Computer Vision/1. Image Classification/3. plant_diseases')
!ls

detect_plant_diseases.ipynb


Import required libraries

In [2]:
import tensorflow as tf
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras import Model
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [3]:
tf.__version__
np.__version__

'1.23.2'

## 1. Load the PlantVillage dataset

In [4]:
import tensorflow_datasets as tfds
dataset, info = tfds.load("plant_village", with_info=True, as_supervised=True)
full_dataset = dataset["train"]

In this step, you load the PlantVillage dataset using TensorFlow Datasets (TFDS). The **tfds.load()** function is used to load the dataset with the following parameters:

*   **"plant_village"**: The name of the dataset to load.
*   **with_info=True**: Indicates that we also want to retrieve the dataset's metadata (e.g., the number of examples, splits, etc.). The metadata is stored in the **info** variable.
*   **as_supervised=True**: Specifies that we want to load the dataset in a supervised format, with separate variables for images and labels.

Once the dataset is loaded, we store the "train" split of the dataset in the **full_dataset** variable. This will be used to create the training and testing datasets in the following step.

## 2. Shuffle and split the dataset into training and testing sets

In [5]:
train_size = int(0.8 * info.splits["train"].num_examples)
test_size = info.splits["train"].num_examples - train_size
full_dataset = full_dataset.shuffle(buffer_size=info.splits["train"].num_examples)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)

In this step, we shuffle and split the "train" split of the PlantVillage dataset into separate training and testing datasets:
*    **train_size = int(0.8 * info.splits["train"].num_examples)**: Calculate the number of examples that will be used for training. We use 80% of the total examples in the "train" split of the dataset for training purposes.
*   **test_size = info.splits["train"].num_examples - train_size**: Calculate the number of examples that will be used for testing. We use the remaining 20% of the total examples in the "train" split of the dataset for testing purposes.
*   **full_dataset = full_dataset.shuffle(buffer_size=info.splits["train"].num_examples)**: Shuffle the entire "train" split of the dataset to ensure the data is randomly ordered. The **buffer_size** parameter determines the number of elements that should be loaded into the buffer for shuffling. Setting it to the total number of examples in the "train" split ensures a uniform shuffle.
*   **train_dataset = full_dataset.take(train_size)**: Create the training dataset by taking the first **train_size** examples from the shuffled dataset. This will correspond to 80% of the total examples in the "train" split.
*   **test_dataset = full_dataset.skip(train_size)**: Create the testing dataset by skipping the first **train_size** examples from the shuffled dataset and taking the remaining examples. This will correspond to the remaining 20% of the total examples in the "train" split.

This step is essential for creating separate training and testing datasets from the original dataset. By shuffling and splitting the data, we can ensure that the model is trained on diverse examples and can be evaluated on an independent test set to assess its performance.

## 3. Set input image shape, number of classes, and batch size

In [6]:
input_shape = (128, 128)
num_classes = 38
batch_size = 16

In this step, we define important parameters that will be used throughout the model building and training process:
*   **input_shape = (128, 128)**: This is a tuple representing the height and width of the input images that the model will process. In our case, we resize the images to 128x128 pixels. This parameter is important because it determines the dimensions of the input tensor that the model will expect during training and evaluation.
*   **num_classes = 38**: This is the number of unique classes in the dataset. In the PlantVillage dataset, there are 38 distinct plant diseases. This parameter is essential for defining the output layer of the model, as it specifies the number of output neurons (each corresponding to a class) in the final layer.
*   **batch_size = 16**: This is the number of examples that will be processed in a single batch during training and evaluation. A smaller batch size can help improve generalization and reduce memory usage, while a larger batch size can lead to faster training. It's important to select an appropriate batch size based on the available resources and the specific problem being solved.

These parameters are essential for defining the model architecture, as well as controlling the training process. Choosing appropriate values for these parameters can significantly impact the model's performance and training efficiency.

## 4. Preprocess the images and labels

In [7]:
def preprocess(image, label):
    # Resize the image to the specified input shape and normalize pixel values to [0, 1]
    image = tf.image.resize(image, input_shape) / 255.0
    return image, label

# Apply preprocessing to the train and test datasets, create batches, and prefetch the data for faster access.

train_dataset = train_dataset.map(preprocess).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In this step, we define a preprocessing function and apply it to the training and testing datasets:
*   **preprocess(image, label)**: This function takes an image and its corresponding label as input. Inside the function, we resize the image to the specified **input_shape** (128x128 pixels in our case) and normalize its pixel values to the range [0, 1] by dividing by 255. The preprocessed image and the original label are then returned.
*   **train_dataset = train_dataset.map(preprocess).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)**: We apply the preprocessing function to the training dataset using the **map** method. Then, we create batches of the specified **batch_size** (16 in our case) using the **batch** method. Finally, we use the **prefetch** method with **tf.data.experimental.AUTOTUNE** to load the data more efficiently during training.
*   **test_dataset = test_dataset.map(preprocess).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)**: We apply the same preprocessing, batching, and prefetching steps to the testing dataset.

Preprocessing the images ensures the model receives consistent input during training and evaluation. Resizing the images to a fixed shape allows the model to process the data more efficiently while normalizing pixel values helps improve the model's training convergence. Batching and prefetching the data can significantly speed up the training process by reducing the time spent loading the data.

## 5. Load the pre-trained MobileNetV2 model without the top classification layer

In [8]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(*input_shape, 3))

In this step, we load the pre-trained MobileNetV2 model as the base model for our image classification task:
*   **weights="imagenet"**: This specifies that we want to use the model weights pre-trained on the ImageNet dataset. Pre-trained weights provide a good starting point for our model, as they have already learned useful features from a large dataset that can be transferred to our specific task.
*   **include_top=False**: This option excludes the top classification layer of the original MobileNetV2 model. By doing so, we can add custom layers on top of the base model to adapt it to our specific problem.
*    **input_shape=(*input_shape, 3)**: This sets the expected input shape of the base model to match the shape of our preprocessed images (128x128 pixels with 3 color channels).

Loading a pre-trained model like MobileNetV2 significantly speeds up the training process and improves its performance, as it has already learned valuable features from a large and diverse dataset. By removing the top classification layer and adding custom layers, we can adapt the base model to our specific task while benefiting from the transfer learning process.

## 6. Add custom layers on top of the pre-trained base model

In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation="softmax")(x)

In this step, custom layers are added on top of the pre-trained base model (MobileNetV2) to tailor it to the specific image classification task using the PlantVillage dataset:
*   **x = base_model.output**: The output of the pre-trained base model is extracted and used as the input for the custom layers.
*   **x = GlobalAveragePooling2D()(x)**: A Global Average Pooling 2D (GAP) layer is added. This layer reduces the spatial dimensions of the feature maps by computing the average value of each feature map. The GAP layer helps reduce the number of parameters in the model and is less prone to overfitting compared to a fully connected layer. It also helps the model adapt to different input sizes.
*   **x = Dense(1024, activation="relu")(x)**: A fully connected (Dense) layer with 1024 units and ReLU (Rectified Linear Unit) activation is added. This layer helps the model learn high-level features specific to the PlantVillage dataset. The ReLU activation function introduces non-linearity into the model, allowing it to learn complex patterns in the data.
*   **x = Dropout(0.5)(x)**: A Dropout layer is added with a dropout rate of 0.5 (50%). During training, this layer randomly sets a fraction (50% in this case) of the input units to 0 at each update, which helps prevent overfitting. The Dropout layer acts as a regularization technique by encouraging the model to learn robust features that are not dependent on specific input units.
*   **predictions = Dense(num_classes, activation="softmax")(x)**: A final output (Dense) layer with num_classes units (38 in this case) and softmax activation is added. The softmax activation function ensures that the output of the model is a probability distribution over the classes, making it suitable for multi-class classification. Each unit in the output layer corresponds to a class in the PlantVillage dataset, and the softmax activation assigns a probability to each class.

## 7. Create the final model

In [10]:
model = Model(inputs=base_model.input, outputs=predictions)

In this step, the final model is created by combining the pre-trained base model (MobileNetV2) and the custom layers added in the previous step.
*   **model = Model(inputs=base_model.input, outputs=predictions)**: This line creates a new Model object, specifying the input and output layers.
*   **inputs=base_model.input**: The input of the final model is set to the input of the pre-trained base model. This ensures that the images fed to the final model are first passed through the pre-trained base model.
*   **outputs=predictions**: The output of the final model is set to the output of the custom layers, which is the predictions variable. This connects the custom layers to the pre-trained base model, creating an end-to-end model architecture that can be used for training and evaluation on the PlantVillage dataset.

## 8. Freeze the base model layers to prevent them from being updated during initial training

In [11]:
for layer in base_model.layers:
  layer.trainable = False

In this step, the layers of the pre-trained base model (MobileNetV2) are frozen to prevent them from being updated during the initial training phase.
*   **for layer in base_model.layers: layer.trainable = False**: This loop iterates over all the layers in the base model and sets their trainable attribute to False. When a layer's trainable attribute is set to False, its weights and biases are not updated during training, effectively "freezing" the layer.

## 9. Compile the model for training

In [12]:
model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In this step, the model is compiled for training by specifying the optimizer, loss function, and evaluation metrics.
*   **optimizer=Adam(learning_rate=0.001)**: The Adam optimizer is chosen for updating the model's weights during training. The learning rate is set to 0.001, which determines the step size taken in the weight update process. The learning rate can affect the model's convergence speed and the quality of the final weights. A smaller learning rate might result in slower convergence but potentially better final weights, while a larger learning rate can lead to faster convergence but possibly less accurate final weights.
*   **loss="sparse_categorical_crossentropy"**: The sparse categorical crossentropy loss function is specified for the multi-class classification task. This loss function measures the difference between the true labels and the predicted probabilities output by the model. It's called "sparse" because the true labels are provided as integers (e.g., 0, 1, 2), rather than one-hot encoded vectors. The goal during training is to minimize this loss, which would result in better classification accuracy.
*   **metrics=["accuracy"]**: The accuracy metric is chosen for evaluating the model's performance during training and validation. Accuracy is the proportion of correctly classified images out of the total number of images. By tracking the accuracy metric, you can monitor the model's progress and see how well it's learning to classify images in the PlantVillage dataset.

## 10. Train the model with the custom layers while keeping the base model layers frozen

In [13]:
history = model.fit(train_dataset, epochs=7, validation_data=test_dataset)

Epoch 1/7


2023-06-20 09:47:46.128335: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2716/2716 [==============================] - 111s 36ms/step - loss: 1.6166 - accuracy: 0.7758 - val_loss: 1.1816 - val_accuracy: 0.8612
Epoch 2/7
2716/2716 [==============================] - 105s 36ms/step - loss: 1.7934 - accuracy: 0.8550 - val_loss: 1.1237 - val_accuracy: 0.8963
Epoch 3/7
2716/2716 [==============================] - 104s 36ms/step - loss: 1.9330 - accuracy: 0.8779 - val_loss: 1.6262 - val_accuracy: 0.8919
Epoch 4/7
2716/2716 [==============================] - 104s 35ms/step - loss: 2.0345 - accuracy: 0.8915 - val_loss: 1.4699 - val_accuracy: 0.9048
Epoch 5/7
2716/2716 [==============================] - 106s 36ms/step - loss: 2.0141 - accuracy: 0.9009 - val_loss: 1.2185 - val_accuracy: 0.9213
Epoch 6/7
2716/2716 [==============================] - 105s 36ms/step - loss: 2.1242 - accuracy: 0.9063 - val_loss: 1.6368 - val_accuracy: 0.9183
Epoch 7/7
2716/2716 [==============================] - 105s 36ms/step - loss: 2.1503 - accuracy: 0.9130 - val_loss: 0.8973 - val_accur

In this step, the model is trained on the PlantVillage dataset using the custom layers, while keeping the base model layers frozen:
*   **train_dataset**: The preprocessed training dataset is provided as input for training. The dataset consists of images and their corresponding labels, which the model will use to update its weights and learn the features specific to the PlantVillage dataset.
*   **epochs=7**: The number of training epochs is set to 7. An epoch is a complete iteration through the entire training dataset. By specifying 7 epochs, the model will go through the entire dataset 7 times, updating its weights and biases with each iteration to minimize the loss function. The number of epochs can affect the model's performance, as too few epochs might lead to underfitting, while too many epochs can cause overfitting. Experimenting with the number of epochs is important to find the best balance for your specific task.
*   **validation_data=test_dataset**: The preprocessed test dataset is provided for validation during training. The validation dataset is not used to update the model's weights but is instead used to evaluate the model's performance on unseen data. By monitoring the model's performance on the validation dataset, you can track how well the model generalizes to new data, helping you to identify potential issues such as overfitting.

The **history** variable stores the training history, including the training and validation loss and accuracy values for each epoch. You can use this information to visualize the training progress and adjust the model or training parameters as needed.

By training the model with the custom layers while keeping the base model layers frozen, you can leverage the pre-trained features from the MobileNetV2 model and learn new high-level features specific to the PlantVillage dataset. This approach allows you to create a powerful image classification model with less training time and computational resources than training a model from scratch.

## 11. Unfreeze all layers of the model

In [14]:
for layer in model.layers[:]:
  layer.trainable = True

In this step, all the layers of the model, including the base model layers, are unfrozen to allow them to be updated during the fine-tuning phase:
*   **for layer in model.layers[:]**: This loop iterates over all layers in the entire model (both the pre-trained base model and custom layers) and sets their **trainable** attribute to **True**.
*   **layer.trainable = True**: When a layer's **trainable** attribute is set to **True**, its weights and biases can be updated during training.

Unfreezing all the layers of the model has several benefits:
*   Fine-tuning: By allowing the base model layers to be updated, the model can further fine-tune its weights on the PlantVillage dataset. This can improve the model's performance, as the pre-trained features can be adapted more specifically to the target dataset.
*   Learning dataset-specific features: By updating the weights of the base model layers, the model can learn dataset-specific features that were not present in the original ImageNet dataset. This can enhance the model's ability to recognize and classify images from the PlantVillage dataset.

It's important to note that when unfreezing the base model layers, using a lower learning rate during training is usually a good idea to avoid overwriting the pre-trained features too quickly. The lower learning rate allows the model to make more minor weight adjustments, preserving the learned features while adapting them to the specific dataset.

## 12. Compile the model with a lower learning rate for fine-tuning

In [15]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In this step, the model is compiled again with a lower learning rate for fine-tuning. This is done to ensure that the pre-trained features in the base model layers are not drastically altered during the fine-tuning phase, allowing the model to maintain its previously learned features while adapting to the specific dataset:
*   **optimizer=Adam(learning_rate=0.0001)**: The learning rate for the Adam optimizer is now set to a lower value (0.0001) than the initial training phase (0.001). A lower learning rate ensures that the model weights are updated with smaller steps, allowing for a more controlled fine-tuning process. This helps preserve the pre-trained features in the base model layers while allowing the model to adapt to the specific PlantVillage dataset.
*   **loss="sparse_categorical_crossentropy"**: The sparse categorical cross-entropy loss function for the multi-class classification task.
*   **metrics=["accuracy"]**: The accuracy metric for evaluating the model's performance during training and validation.

By compiling the model with a lower learning rate for fine-tuning, you can better control the weight updates and ensure that the pre-trained features are not drastically altered, potentially leading to a more accurate and robust model.

## 13. Fine-tune the entire model with a lower learning rate

In [16]:
history_finetune = model.fit(train_dataset, epochs=4, validation_data=test_dataset)

Epoch 1/4
2716/2716 [==============================] - 279s 97ms/step - loss: 3.7241 - accuracy: 0.8771 - val_loss: 2.3005 - val_accuracy: 0.8960
Epoch 2/4
2716/2716 [==============================] - 268s 96ms/step - loss: 1.1601 - accuracy: 0.9329 - val_loss: 0.6320 - val_accuracy: 0.9564
Epoch 3/4
2716/2716 [==============================] - 266s 95ms/step - loss: 0.8333 - accuracy: 0.9423 - val_loss: 0.4277 - val_accuracy: 0.9592
Epoch 4/4
2716/2716 [==============================] - 266s 95ms/step - loss: 0.6708 - accuracy: 0.9488 - val_loss: 0.5861 - val_accuracy: 0.9550


In this step, the entire model, including the previously frozen base model layers, is fine-tuned with a lower learning rate:
*   **train_dataset**: The preprocessed training dataset is provided as input for fine-tuning. The dataset consists of images and their corresponding labels, which the model will use to update its weights and learn the features specific to the PlantVillage dataset.
*   **epochs=4**: The number of fine-tuning epochs is set to 4. The model will go through the entire dataset 4 times, updating its weights and biases with each iteration. This is a separate count from the initial training phase, meaning the model will have trained for a total of 11 epochs (7 initial + 4 fine-tuning). The choice of the number of fine-tuning epochs depends on the specific problem and dataset, and it may require experimentation to find the best balance between underfitting and overfitting.
*   **validation_data=test_dataset**: The preprocessed test dataset is provided for validation during fine-tuning. The validation dataset is not used to update the model's weights but to evaluate the model's performance on unseen data. Monitoring the model's performance on the validation dataset can help you identify potential issues, such as overfitting or underfitting.

The history_finetune variable stores the fine-tuning history, which includes the training and validation loss and accuracy values for each epoch. You can use this information to visualize the fine-tuning progress and make adjustments to the model or training parameters as needed.

By fine-tuning the entire model with a lower learning rate, you can further adapt the pre-trained features to the PlantVillage dataset, potentially improving the model's performance and generalization to unseen data.

## 14. Evaluate the model on the test dataset

In [17]:
model.evaluate(test_dataset)

679/679 [==============================] - 25s 25ms/step - loss: 0.5849 - accuracy: 0.9544


[0.5849285125732422, 0.9544241428375244]

In this step, the fine-tuned model is evaluated on the test dataset. The evaluate method computes the model's performance metrics (loss and accuracy in this case) on the test dataset. The test dataset consists of images and labels that the model has not seen during training or fine-tuning, providing an unbiased evaluation of the model's generalization ability on new, unseen data.

The results you provided indicate the following:
*   **loss: 0.0559**: The model's computed loss value on the test dataset is 0.0559. The loss value is obtained using the sparse categorical cross-entropy loss function, which measures the difference between the true labels and the model's predicted probabilities for each class. A lower loss value indicates better performance and better alignment between the model's predictions and the true labels.
*   **accuracy: 0.9810**: The model's accuracy on the test dataset is 0.9810 (98.10%). This means that the model correctly classified 98.10% of the images in the test dataset. Accuracy is a commonly used metric to measure the performance of a classification model. The higher the accuracy, the better the model's ability to correctly classify images.

Your results (loss: 0.0559, accuracy: 0.9810) indicate that the fine-tuned model performs well on the test dataset, achieving a high accuracy of 98.10%. This suggests that the model has successfully learned to recognize and classify images from the PlantVillage dataset and is expected to generalize well to new, unseen data.

# 15. Save the model

In [18]:
model.save("./[2023-06-20T10:19:00]_plant_diseases")

INFO:tensorflow:Assets written to: ./[2023-06-20T10:19:00]_plant_diseases/assets


INFO:tensorflow:Assets written to: ./[2023-06-20T10:19:00]_plant_diseases/assets
